In [4]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from collections import defaultdict

In [5]:
# Divide the data into two time frames
ref_date = datetime.datetime(day=1, month=1, year=1997)
ref_date

datetime.datetime(1997, 1, 1, 0, 0)

In [6]:
# Define path to files
path_to_files = "../berkabank/elaboration/"


# Load the data
files = ["account", "card", "client", "disp", "district", "loan", "order", "trans"]
data = {file: pd.read_csv(f"{path_to_files}{file}.csv") for file in files}

# Convert dates to datetime
data = {
    key: df.apply(lambda col: pd.to_datetime(col) if "date" in col.name else col)
    for key, df in data.items()
}

In [7]:
# Collect into a dict dataframes name: columns with date substring
date_columns = {
    key: [col for col in df.columns if "date" in col] for key, df in data.items()
}
date_columns

{'account': ['account_creation_date'],
 'card': ['card_issued_date'],
 'client': [],
 'disp': [],
 'district': [],
 'loan': ['loan_date', 'estimated_loan_end_date'],
 'order': [],
 'trans': ['transaction_date']}

In [8]:
# Filter transactions before and after the reference date
transaction_t0 = data["trans"].loc[data["trans"]["transaction_date"] < ref_date, :]
transaction_t1 = data["trans"].loc[data["trans"]["transaction_date"] >= ref_date, :]

# Filter accounts before and after the reference date
account_t0 = data["account"].loc[data["account"]["account_creation_date"] < ref_date, :]
account_t1 = data["account"].loc[
    data["account"]["account_creation_date"] >= ref_date, :
]

# Filter cards before and after the reference date
card_t0 = data["card"].loc[data["card"]["card_issued_date"] < ref_date, :]
card_t1 = data["card"].loc[data["card"]["card_issued_date"] >= ref_date, :]

In [16]:
# Cast dates to date 
transaction_t0["transaction_date"] = pd.to_datetime(transaction_t0["transaction_date"]).dt.date
transaction_t1["transaction_date"] = pd.to_datetime(transaction_t1["transaction_date"]).dt.date
account_t0["account_creation_date"] = pd.to_datetime(account_t0["account_creation_date"]).dt.date
account_t1["account_creation_date"] = pd.to_datetime(account_t1["account_creation_date"]).dt.date
card_t0["card_issued_date"] = pd.to_datetime(card_t0["card_issued_date"]).dt.date
card_t1["card_issued_date"] = pd.to_datetime(card_t1["card_issued_date"]).dt.date   

/var/folders/2v/jln6f4j122qcvxp2jjr6cclm0000gn/T/ipykernel_97367/475800764.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_t0["transaction_date"] = pd.to_datetime(transaction_t0["transaction_date"]).dt.date
/var/folders/2v/jln6f4j122qcvxp2jjr6cclm0000gn/T/ipykernel_97367/475800764.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_t1["transaction_date"] = pd.to_datetime(transaction_t1["transaction_date"]).dt.date
/var/folders/2v/jln6f4j122qcvxp2jjr6cclm0000gn/T/ipykernel_97367/4

In [17]:
# Save the data
path_to_files = "../berkabank/primary/"
transaction_t0.to_csv(f"{path_to_files}transactions.csv", index=False)
account_t0.to_csv(f"{path_to_files}accounts.csv", index=False)
card_t0.to_csv(f"{path_to_files}cards.csv", index=False)

In [18]:
account_t0.shape, transaction_t0.shape, card_t0.shape

((3602, 4), (449634, 8), (201, 4))

In [22]:
transaction_t0.dtypes

trans_id                               int64
account_id                             int64
transaction_date                      object
transaction_type                      object
transaction_operation                 object
transaction_amount                   float64
account_balance_after_transaction    float64
k_symbol                              object
dtype: object